# Analisis de Sentimientos en reseñas de Steam.
**Nombre**
**Fecha**
**Materia**

## Descripcion
> text

## Importacion de librerias


Aquí se importan las librerías esenciales para el procesamiento de lenguaje natural, análisis de datos y visualización. Utilizaremos nltk para el análisis de texto, pandas para la manipulación de dataframes, matplotlib y seaborn para graficar, y otras librerías como re y random para tareas específicas.

- `nltk` : Para el analisis de texto
- `pandas` : Para Para la manipulacion de dataframes
- `matplotlib y seaborn` : Para graficar
- `re` : Para expresiones regulares

In [1]:
# Importación de librerías para Procesamiento de Lenguaje Natural
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
import random
import requests

from nltk.corpus import stopwords
from nltk import ngrams

# Descargar recursos de nltk, como stopwords
#nltk.download('punkt')
#nltk.download('stopwords')

# Confirmación de importación
print("Librerías importadas correctamente para el procesamiento de lenguaje natural.")

Librerías importadas correctamente para el procesamiento de lenguaje natural.


## Importacion de Datos de Steam
En esta parte se explica como obtener las reseñas de Steam. Se pueden extraer mediante la API de Steam, utilizando el siguiente endpoint:

`GET https://store.steampowered.com/appreviews/<id_de_aplicación>?json=1`
Se indican algunos parametros utiles como lo son:
- `filter`
- `language`
- `day_range`
- `cursor`
- `review_type`
- `num_per_page`
- `filter_offtopic_activity`

Para saber mas de los parametros puedes ir a [Steam API](https://partner.steamgames.com/doc/store/getreviews)

En esta parte se hace la peticion del endpoint mediante el `id del juego` con los parametros adecuados (En este caso, se extraen 100 reseñas por pagina en lenguaje de español).
La respuesta se presenta en estructura JSON devuelto y psoteriormente se carga en un DataFrame de *Pandas* para hacer el análisis

In [2]:
# URL de la API de Steam para obtener reseñas en español (100 reseñas)
id_game = "2246340"             # Monster Hunter Wilds = 2246340
language = "spanish"            # Idioma de las reseñas
num_per_page = "100"            # Número de reseñas por página
url = f"https://store.steampowered.com/appreviews/{id_game}?json=1&language={language}&num_per_page={num_per_page}"


# Realizar la solicitud a la API
response = requests.get(url)
data = response.json()

# Extraer las reseñas del JSON
reviews = data.get('reviews', [])

# Crear un DataFrame a partir de las reseñas
df_reviews = pd.DataFrame(reviews)

# Mostrar información básica del DataFrame
print("Información del DataFrame:")
print(df_reviews.info())


Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   recommendationid             100 non-null    object
 1   author                       100 non-null    object
 2   language                     100 non-null    object
 3   review                       100 non-null    object
 4   timestamp_created            100 non-null    int64 
 5   timestamp_updated            100 non-null    int64 
 6   voted_up                     100 non-null    bool  
 7   votes_up                     100 non-null    int64 
 8   votes_funny                  100 non-null    int64 
 9   weighted_vote_score          100 non-null    object
 10  comment_count                100 non-null    int64 
 11  steam_purchase               100 non-null    bool  
 12  received_for_free            100 non-null    bool  
 13  written_d

In [ ]:
# Mostar las primeras filas del DataFrame completo
print("\nPrimeras filas del DataFrame completo:")
df_reviews.head(5)


Primeras filas del DataFrame completo:


,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck
0,189321117,"{'steamid': '76561198138131071', 'num_games_ow...",spanish,"AMO el juego, es demasiado bueno y la verdad e...",1741064282,1741840734,False,135,6,0.832816362380981445,0,True,False,False,False
1,188940182,"{'steamid': '76561198049254557', 'num_games_ow...",spanish,Basta ya. Basta de sacar productos incompletos...,1740743526,1740743526,False,79,3,0.70836329460144043,0,True,False,False,False
2,189266847,"{'steamid': '76561198066270836', 'num_games_ow...",spanish,Uno de los mejores juegos de la historia sin d...,1741010357,1741010357,True,23,2,0.706308245658874512,1,True,False,False,False
3,189254007,"{'steamid': '76561198010057144', 'num_games_ow...",spanish,"As a Monster Hunter game, it's absolutely supe...",1740997022,1740997022,False,48,3,0.693980157375335693,5,True,False,False,False
4,188929055,"{'steamid': '76561198417230907', 'num_games_ow...",spanish,el juego de la banana esta mejor optimizado \n...,1740735215,1740751024,True,30,0,0.71985316276550293,0,True,False,False,False


In [ ]:
# Prueba solo comentarios
# DataFrame para unicamente la columna de review y salga completamente el texto
df_reviewsComments = df_reviews[['review']]
df_reviewsComments.head(5)

## Limpieza de texto (Stopwords)
En esta seccion se muestra como limpiar el texto de las reseñas, removiendo stopwords en español para preparar el analisis. Se utilizan las stopwords de *nltk*


In [ ]:
# Definir las stopwords en español
stop_words = set(stopwords.words('spanish'))

# Funcion para limpiar una reseña eliminando con stopwords
review = df_reviews['review'][0]                                    #Tomando la primera reseña
words = nltk.word_tokenize(review.lower(), language='spanish')      #Tokenizando la reseña
clean_words = [word for word in words if word not in stop_words and word.isalpha()]

print("Reseña original (Antes de hacer 'stopwords'):")
print(review)
print("\nReseña limpia (sin stopwords):")
print(" ".join(clean_words))
#Almacenarlo en un dataframe



Reseña original (Antes de hacer 'stopwords'):
AMO el juego, es demasiado bueno y la verdad es que superó todas mi expectativas,
PERO, tiene una de las peores optimizaciones que he visto en mi vida, necesita parchearse URGENTE

Reseña limpia (sin stopwords):
amo juego demasiado bueno verdad superó todas expectativas peores optimizaciones visto vida necesita parchearse urgente


## Tokenizacion
En esta seccion ilustra el proceso de tokenizacion, dividiendo el texto en palabras (tokens) para facilitar el analisis posterior


In [ ]:
# Tokenización de una reseña (utilizando la misma reseña de ejemplo)
tokens = nltk.word_tokenize(review, language='spanish')
print("Tokens de la reseña:")
print(tokens)


Tokens de la reseña:
['AMO', 'el', 'juego', ',', 'es', 'demasiado', 'bueno', 'y', 'la', 'verdad', 'es', 'que', 'superó', 'todas', 'mi', 'expectativas', ',', 'PERO', ',', 'tiene', 'una', 'de', 'las', 'peores', 'optimizaciones', 'que', 'he', 'visto', 'en', 'mi', 'vida', ',', 'necesita', 'parchearse', 'URGENTE']


## Obtener la reseña mas larga
A continuacion se extrae de las reseñas la que tenga mayor cantidad de caracteres o palabras.

In [6]:
# Calcular la longitud de cada reseña (por número de caracteres)
df_reviews['review_length'] = df_reviews['review'].apply(len)

# Seleccionar la reseña con mayor longitud
longest_review = df_reviews.loc[df_reviews['review_length'].idxmax()]['review']

print("La reseña más larga es:")
print(longest_review)


La reseña más larga es:
Gráficamente - es muy bello. apreciar los cambios en los biomas de abundante, sequía o el ambiente característico de la zona, ver como se desenvuelve en la zona los monstruos en ella. el como la fauna aporta en estos bellos lugares, lo juego en calidad media llegando a baja, la 3060 laptop no quiere mas. de rendimiento es raro puedo estar muy estable cazando pero entro a los campamentos y todo se arruina, se puede decir que va mas o menos. tengo que ser auto critico no puedo 
pedirle mucho a una laptop del 2021.

Historia - Me parece interesante el como se ingresa a el tema de las civilizaciones anteriores [spoiler] Sobretodo en las clonaciones y mejoras genéticas que lograron, o las forma de generar energía infinita mediante a un liquido [/spoiler]. pero la historia donde te conviertes en el pokemon del gremio o si lo ves de forma mas bonita el Alucard de integra Hellsing se 
siente raro, aunque eres lo maximo ya que nadie duda de que eres una maquina de matar,

## N-gramas
N-Gramas (Bi-Gramas, Tri-Gramas) a partir de las reseñas del videojuegos, para identificar secuencia frecuentes de palabras
Se utiliza la libreria de `from nltk import ngrams` para poder hacer los n-gramas

In [ ]:
# Funcion para extraer n-gramas de una reseña
def get_ngrams(text, n=2):
    token = nltk.word_tokenize(text.lower(), language='spanish')
    return list(ngrams(token, n))

# Extraer n-gramas (Bi-gramas) de una reseña Aleatoria de (1-100)
bigrams = get_ngrams(review, n=2)
print(f"Bi-gramas de la reseña aleatoria:")
print(bigrams)


Bi-gramas de la reseña aleatoria:
[('amo', 'el'), ('el', 'juego'), ('juego', ','), (',', 'es'), ('es', 'demasiado'), ('demasiado', 'bueno'), ('bueno', 'y'), ('y', 'la'), ('la', 'verdad'), ('verdad', 'es'), ('es', 'que'), ('que', 'superó'), ('superó', 'todas'), ('todas', 'mi'), ('mi', 'expectativas'), ('expectativas', ','), (',', 'pero'), ('pero', ','), (',', 'tiene'), ('tiene', 'una'), ('una', 'de'), ('de', 'las'), ('las', 'peores'), ('peores', 'optimizaciones'), ('optimizaciones', 'que'), ('que', 'he'), ('he', 'visto'), ('visto', 'en'), ('en', 'mi'), ('mi', 'vida'), ('vida', ','), (',', 'necesita'), ('necesita', 'parchearse'), ('parchearse', 'urgente')]
